# Results

## Necessary imports

In [1]:
import os
import sys
import warnings
warnings.filterwarnings("ignore")

In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

## Check Database, Tables and table columns

In [33]:
spark.catalog.listDatabases()

[Database(name='default', description='Default Hive database', locationUri='file:/home/big/Desktop/Aulas/Projeto/spark-warehouse')]

In [4]:
spark.catalog.listTables(dbName="default")

[Table(name='datatable', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='itemsrecommendationtable', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='itemstable', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='userrecommendationtable', database='default', description=None, tableType='MANAGED', isTemporary=False)]

In [5]:
spark.sql("USE default")

DataFrame[]

In [6]:
spark.catalog.listColumns("datatable")

[Column(name='Reviewer', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='Item', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='Rating', description=None, dataType='double', nullable=True, isPartition=False, isBucket=False),
 Column(name='ReviewerID', description=None, dataType='double', nullable=True, isPartition=False, isBucket=False),
 Column(name='ItemID', description=None, dataType='double', nullable=True, isPartition=False, isBucket=False)]

In [7]:
spark.sql("SELECT * FROM datatable").show(10,truncate=False)

+--------------+----------+------+----------+-------+
|Reviewer      |Item      |Rating|ReviewerID|ItemID |
+--------------+----------+------+----------+-------+
|A1BU91W99GF2RW|B00C776C3I|4.0   |8439.0    |67844.0|
|A3IDGI3N7ZF054|B00C77M5RK|5.0   |54329.0   |67845.0|
|A1YWK7JOEG5H96|B00C78F3FU|4.0   |21901.0   |67846.0|
|A1Z3ZBLQELV9Z8|B00C78G3G8|5.0   |22029.0   |67847.0|
|AXQYR6U66GR54 |B00C78K8NW|4.0   |79886.0   |67848.0|
|A2MFTD1SV25S9V|B00C78W0ME|5.0   |35581.0   |67849.0|
|A3E3HHHYYX4AMS|B00C7A8WWO|5.0   |51845.0   |67850.0|
|A1L86G7UESAN0S|B00C7ACXJW|5.0   |13918.0   |67851.0|
|A1EX0MR65MFI8C|B00C7BQSUQ|5.0   |10231.0   |67852.0|
|A33QAS2S0I9WDZ|B00C7BZYRO|1.0   |45780.0   |67853.0|
+--------------+----------+------+----------+-------+
only showing top 10 rows



## Show recommendations

#### For all users and for all items

In [8]:
spark.sql("SELECT * FROM userrecommendationtable").show(10,truncate=False)

+----------+-----------------------------------------------------------------------------------------+
|ReviewerID|recommendations                                                                          |
+----------+-----------------------------------------------------------------------------------------+
|65        |[[325, 8.40369], [234, 8.390948], [1192, 7.315639], [26, 6.9698887], [141, 6.85389]]     |
|458       |[[99, 8.441162], [36, 7.518686], [181, 7.00895], [566, 6.839488], [1325, 6.703174]]      |
|879       |[[63, 10.026212], [78, 8.0875845], [96, 7.506577], [890, 7.142868], [1965, 6.871914]]    |
|883       |[[135, 8.627517], [240, 7.912772], [193, 7.7639256], [681, 7.35819], [118, 7.3522406]]   |
|1223      |[[598, 8.864721], [1205, 8.101884], [15, 7.980553], [75, 7.199368], [101, 6.756503]]     |
|1977      |[[92, 5.845594], [571, 5.1056643], [39, 5.0836368], [2215, 4.9989033], [393, 4.9713798]] |
|2096      |[[623, 8.079464], [515, 7.7680893], [427, 7.344784], [41, 7.2

In [9]:
spark.sql("SELECT * FROM itemsrecommendationtable").show(10,truncate=False)

+------+--------------------------------------------------------------------------------------------------+
|ItemID|recommendations                                                                                   |
+------+--------------------------------------------------------------------------------------------------+
|65    |[[62650, 9.562832], [23183, 9.4881935], [26528, 9.393593], [75187, 9.239661], [42, 9.111745]]     |
|458   |[[3151, 8.219126], [75092, 8.192369], [69239, 8.181688], [49058, 8.134698], [68769, 8.115892]]    |
|879   |[[41002, 6.4863915], [62791, 6.484882], [61542, 6.378982], [14904, 6.3169246], [37309, 6.2771187]]|
|883   |[[35543, 4.997857], [71591, 4.6043515], [40283, 4.573918], [53758, 4.5631623], [80912, 4.516834]] |
|1223  |[[891, 7.059295], [52290, 6.862376], [26824, 6.827621], [4439, 6.825762], [45409, 6.8136907]]     |
|1977  |[[28297, 3.9972646], [58198, 3.7235363], [73892, 3.7229476], [21758, 3.665168], [2506, 3.6521032]]|
|2096  |[[35286, 5.289352], 

#### For a single user and a single item

In [10]:
user = 54329
item = 67844

In [31]:
spark.sql("SELECT datatable.Item, itemsrecommendationtable.recommendations FROM itemsrecommendationtable INNER JOIN datatable ON itemsrecommendationtable.ItemID = datatable.ItemID WHERE datatable.ItemID ="+str(item)+" LIMIT 1").show(1)

+----------+--------------------+
|      Item|     recommendations|
+----------+--------------------+
|B008JGSM6G|[[76237, 9.355507...|
+----------+--------------------+



In [32]:
spark.sql("SELECT datatable.Reviewer, userrecommendationtable.recommendations FROM userrecommendationtable INNER JOIN datatable ON userrecommendationtable.ReviewerID = datatable.ReviewerID WHERE datatable.ReviewerID ="+str(user)+" LIMIT 1").show(1)

+--------------+--------------------+
|      Reviewer|     recommendations|
+--------------+--------------------+
|A1Y5IF71BX7URN|[[118, 9.143514],...|
+--------------+--------------------+

